In [1]:
import os
import json
import pandas as pd

import traceback

from langchain.llms import HuggingFaceHub

import warnings
warnings.filterwarnings(action="ignore")

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2

In [3]:
API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN_NEW")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=API_TOKEN,
    model_kwargs={
        "max_length": 100,  # Limit response length
        "temperature": 0.7,  # Control creativity
        "top_p": 0.9        # Nucleus sampling
    }
)

Designing the Prompt

In [4]:
RESPONSE_JSON = {
    "1": {
        "MCQ" : "Multiple Choice Questions",
        "Options":
        {
            "A": "Option A",
            "B": "Option B",
            "C": "Option C",
            "D": "Option D"
        },
        "Correct": "Correct Answer",
        },
    "2": {
        "MCQ" : "Multiple Choice Questions",
        "Options":
        {
            "A": "Option A",
            "B": "Option B",
            "C": "Option C",
            "D": "Option D"
        },
        "Correct": "Correct Answer",
        },
    "3": {
        "MCQ" : "Multiple Choice Questions",
        "Options":
        {
            "A": "Option A",
            "B": "Option B",
            "C": "Option C",
            "D": "Option D"
        },
        "Correct": "Correct Answer",
        }
  }

In [5]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} MCQs for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [6]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

Creating chain

In [7]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    verbose=True,
    output_key="quiz"
)

Evaluating the quality of quiz generated

In [8]:
TEMPLATE2 = """
You are an expert Englist Grammarian and writer. Given the MCQ for {subject} students \ 
Evaluate the MCQ and check if the MCQ is correct or not. \
Make sure to check the MCQ and options are correct and not repeated. \
Update the quiz inf necassary. \
{quiz}
"""

quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    verbose=True,
    output_key="review"
)

Using Sequential Chain

In [9]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

Based on this text, we wil generate the quiz

In [10]:
file_path = "D:\WORK STUDY\PROJECTS\MCQ Generator\experiment\data.txt"

with open(file_path, "r") as file:
    text = file.read()
print(text)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approxima

In [11]:
response = generate_evaluate_chain(
    {'text':text,
    'number':4,
    'subject':"Machine Learning",
    'tone':"Hard",
    'response_json':json.dumps(RESPONSE_JSON)
    }
)

print(response.keys())



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on explo

In [13]:
response.get("quiz")

'\nText: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, pro

In [16]:
quiz = response['quiz']
print(type(quiz))
print(quiz)

<class 'str'>

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoin

In [47]:
ind = quiz.index("### RESPONSE_JSON")
quiz_dict = quiz[ind+13:].split("RESPONSE_JSON")[1]
quiz_dict_new = quiz_dict.replace("###", "")
print(quiz_dict_new)


{
"1": {
"MCQ": "Which field of study in artificial intelligence is concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data?",
"Options": {
"A": "Computer Vision",
"B": "Deep Learning",
"C": "Machine Learning",
"D": "Natural Language Processing"
},
"Correct": "C"
},
"2": {
"MCQ": "The application of machine learning to business problems is known as:",
"Options": {
"A": "Predictive Analytics",
"B": "Deep Learning",
"C": "Neural Networks",
"D": "Statistics"
},
"Correct": "A"
},
"3": {
"MCQ": "Which mathematical optimization method is used as a foundation for machine learning?",
"Options": {
"A": "Probability Theory",
"B": "Mathematical Programming",
"C": "Linear Algebra",
"D": "Calculus"
},
"Correct": "B"
},
"4": {
"MCQ": "Who coined the term machine learning in 1959?",
"Options": {
"A": "Geoffrey Hinton",
"B": "Arthur Samuel",
"C": "Yann LeCun",
"D": "Russell Tesman"
},
"Correct": "B"
}
}

## Here is the solution
```json

In [48]:
sol_index = quiz_dict_new.index("## Here is the solution")
quiz_dict_new = quiz_dict_new[:sol_index]
print(quiz_dict_new)


{
"1": {
"MCQ": "Which field of study in artificial intelligence is concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data?",
"Options": {
"A": "Computer Vision",
"B": "Deep Learning",
"C": "Machine Learning",
"D": "Natural Language Processing"
},
"Correct": "C"
},
"2": {
"MCQ": "The application of machine learning to business problems is known as:",
"Options": {
"A": "Predictive Analytics",
"B": "Deep Learning",
"C": "Neural Networks",
"D": "Statistics"
},
"Correct": "A"
},
"3": {
"MCQ": "Which mathematical optimization method is used as a foundation for machine learning?",
"Options": {
"A": "Probability Theory",
"B": "Mathematical Programming",
"C": "Linear Algebra",
"D": "Calculus"
},
"Correct": "B"
},
"4": {
"MCQ": "Who coined the term machine learning in 1959?",
"Options": {
"A": "Geoffrey Hinton",
"B": "Arthur Samuel",
"C": "Yann LeCun",
"D": "Russell Tesman"
},
"Correct": "B"
}
}




In [51]:
quiz = json.loads(quiz_dict_new)
print(type(quiz))
print(quiz.keys(), quiz.values())

<class 'dict'>
dict_keys(['1', '2', '3', '4']) dict_values([{'MCQ': 'Which field of study in artificial intelligence is concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data?', 'Options': {'A': 'Computer Vision', 'B': 'Deep Learning', 'C': 'Machine Learning', 'D': 'Natural Language Processing'}, 'Correct': 'C'}, {'MCQ': 'The application of machine learning to business problems is known as:', 'Options': {'A': 'Predictive Analytics', 'B': 'Deep Learning', 'C': 'Neural Networks', 'D': 'Statistics'}, 'Correct': 'A'}, {'MCQ': 'Which mathematical optimization method is used as a foundation for machine learning?', 'Options': {'A': 'Probability Theory', 'B': 'Mathematical Programming', 'C': 'Linear Algebra', 'D': 'Calculus'}, 'Correct': 'B'}, {'MCQ': 'Who coined the term machine learning in 1959?', 'Options': {'A': 'Geoffrey Hinton', 'B': 'Arthur Samuel', 'C': 'Yann LeCun', 'D': 'Russell Tesman'}, 'Correct': 'B'}])


In [52]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['MCQ']
    options = " | ".join(
        [
            f"{option}: {value['Options'][option]}" for option in value['Options']
        ]
        )
    correct = value['Correct']
    quiz_table_data.append({
        'MCQ': mcq,
        'Options': options,
        'Correct': correct
    })
quiz_df = pd.DataFrame(data=quiz_table_data)
quiz_df.to_csv("quiz.csv", index=False)